In [1]:
import os
import sys

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn.functional as F

from environments.composition import CompositionGrid
from models.embedding_model import LearnableEmbedding
from models.action_network import LowerPolicyTrainer
from environments.pomdp_config import *
from models.abstract_state_network import AbstractStateNetwork, AbstractStateDataGenerator

In [2]:
###################
# CONSTANTS
###################

device = torch.device("mps")
HYPER_EPOCHS = 50
BATCH_SIZE = 1
MAX_TIMESTEPS = 20
LOWER_STATE_MODEL_PATH = "/Users/vsathish/Documents/Quals/saved_models/pomdp/oct_25_run_1_embedding.state"
LOWER_ACTION_MODEL_PATH = "/Users/vsathish/Documents/Quals/saved_models/action_network/dec_6_run_1_embedding.state"
HIGHER_STATE_MODEL_PATH = "/Users/vsathish/Documents/Quals/saved_models/pomdp/oct_25_run_1_embedding.state"

# Data Generation Constants
COMPOSITION_CONFIG = composite_config1
BASE_CONFIGS = [c1, c2]
NUM_SAMPLES = 1000

# Define env
env = CompositionGrid(COMPOSITION_CONFIG)
env.plot_board()


5 9
5 5 9 5
In mapping function :  1 2
{(0, 0): [array([1., 0.])], (0, 1): [array([1., 0.])], (0, 2): [array([1., 0.])], (0, 3): [array([1., 0.])], (0, 4): [array([1., 0.]), array([0., 1.])], (1, 0): [array([1., 0.])], (1, 1): [array([1., 0.])], (1, 2): [array([1., 0.])], (1, 3): [array([1., 0.])], (1, 4): [array([1., 0.]), array([0., 1.])], (2, 0): [array([1., 0.])], (2, 1): [array([1., 0.])], (2, 2): [array([1., 0.])], (2, 3): [array([1., 0.])], (2, 4): [array([1., 0.]), array([0., 1.])], (3, 0): [array([1., 0.])], (3, 1): [array([1., 0.])], (3, 2): [array([1., 0.])], (3, 3): [array([1., 0.])], (3, 4): [array([1., 0.]), array([0., 1.])], (4, 0): [array([1., 0.])], (4, 1): [array([1., 0.])], (4, 2): [array([1., 0.])], (4, 3): [array([1., 0.])], (4, 4): [array([1., 0.]), array([0., 1.])], (0, 5): [array([0., 1.])], (0, 6): [array([0., 1.])], (0, 7): [array([0., 1.])], (0, 8): [array([0., 1.])], (1, 5): [array([0., 1.])], (1, 6): [array([0., 1.])], (1, 7): [array([0., 1.])], (1, 8): [ar

In [3]:
# Define lower state model
lower_state_model = LearnableEmbedding(device, BATCH_SIZE, timesteps=8).to(device)
try:
    lower_state_model.load_state_dict(torch.load(LOWER_STATE_MODEL_PATH))
    print("################## LOAD SUCCESS #################")
except:
    print("################## NOPE #######################")

# Define lower action model
lower_action_model = LowerPolicyTrainer(device, BATCH_SIZE, MAX_TIMESTEPS).to(device)
try:
    lower_action_model.load_state_dict(torch.load(LOWER_ACTION_MODEL_PATH))
except:
    print("COULD NOT LOAD ACTION NETWORK")

# Define higher state model
higher_state_model = AbstractStateNetwork(9, COMPOSITION_CONFIG["num_blocks"]).to(device)
try:
    higher_state_model.load_state_dict(torch.load(HIGHER_STATE_MODEL_PATH))
except:
    print("COULD NOT LOAD HIGHER STATE NETWORK")


################## LOAD SUCCESS #################
COULD NOT LOAD HIGHER STATE NETWORK


In [4]:
# Define data generator
data_gen = AbstractStateDataGenerator(COMPOSITION_CONFIG, BASE_CONFIGS, lower_state_model, lower_action_model, device)
x_train, y_train = data_gen.generate_data(NUM_SAMPLES)

5 9
5 5 9 5
In mapping function :  1 2
{(0, 0): [array([1., 0.])], (0, 1): [array([1., 0.])], (0, 2): [array([1., 0.])], (0, 3): [array([1., 0.])], (0, 4): [array([1., 0.]), array([0., 1.])], (1, 0): [array([1., 0.])], (1, 1): [array([1., 0.])], (1, 2): [array([1., 0.])], (1, 3): [array([1., 0.])], (1, 4): [array([1., 0.]), array([0., 1.])], (2, 0): [array([1., 0.])], (2, 1): [array([1., 0.])], (2, 2): [array([1., 0.])], (2, 3): [array([1., 0.])], (2, 4): [array([1., 0.]), array([0., 1.])], (3, 0): [array([1., 0.])], (3, 1): [array([1., 0.])], (3, 2): [array([1., 0.])], (3, 3): [array([1., 0.])], (3, 4): [array([1., 0.]), array([0., 1.])], (4, 0): [array([1., 0.])], (4, 1): [array([1., 0.])], (4, 2): [array([1., 0.])], (4, 3): [array([1., 0.])], (4, 4): [array([1., 0.]), array([0., 1.])], (0, 5): [array([0., 1.])], (0, 6): [array([0., 1.])], (0, 7): [array([0., 1.])], (0, 8): [array([0., 1.])], (1, 5): [array([0., 1.])], (1, 6): [array([0., 1.])], (1, 7): [array([0., 1.])], (1, 8): [ar

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)